In [2]:
from tqdm import tqdm
import pandas as pd
import requests as req 
from bs4 import BeautifulSoup as bs
import re

### 학습 목표
- 영화 리뷰데이터 수집 (블로그) 할수 있ㄷ ㅏ
- 수집한 데이터를 시각화 하기위한 전처리를 할수있다
- 워드 클라우드를 생성할수있따 

In [3]:
url='https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ssc=tab.nx.all&query=%EC%95%84%EB%B0%94%ED%83%803+%EB%A6%AC%EB%B7%B0&oquery=%EC%95%84%EB%B0%94%ED%83%802+%EB%A6%AC%EB%B7%B0&tqi=jUsNTwqos5ossi2ZKPZ-342894&ackey=74e97f8t'
print(req.get(url))
new=req.get(url)
#<Response [406]> 멜론에서 브라우저가 아닌 코드로 접근한것을 인지 -> 접근 불가 
chart= bs(new.text,'lxml')

<Response [200]>


In [16]:
ur=chart.select(".list_title>a")

In [17]:
chart.select(".list_title>a")[0].get('href')

'https://blog.naver.com/syunni1225/224115606045'

### 프로세스
1. 영화리뷰가 작성된 블로그 제목 및 주소 수집
2. 리부가 작성된 블로그의 리뷰내용 수집 ()
3. 여러개의 블로그 리뷰내용 수집
4. 수집한 텍스트데이터 전처리
5. 워드 클라우드 생성하기

In [18]:
ti=chart.select(".list_title>a>span")

In [19]:
blog_lst=[]
blog_title=[]
for i in range(len(ur)):
    blog_lst.append(ur[i].get('href'))
    blog_title.append(ti[i].text)
blog_lst   


['https://blog.naver.com/syunni1225/224115606045',
 'https://blog.naver.com/kaminari2002/224114898643',
 'https://blog.naver.com/seok9c/224115025215',
 'https://blog.naver.com/duswns202/224106859517',
 'https://blog.naver.com/dudghkrkwhgdms/224115758710',
 'https://blog.naver.com/blac_beat/224128473123',
 'https://blog.naver.com/realnogun/224113577463',
 'https://blog.naver.com/c106507/224111497322',
 'https://blog.naver.com/sta-tio/224114494844',
 'https://blog.naver.com/stmisn/224091045516']

In [20]:
blog_title

['아바타: 불과 재(Avatar: Fire and Ash, 2025) 영화 정보... ',
 '<아바타3: 불과 재> 4Dx3D후기 :: 그래도 극장에서 봐야하는 영화.... ',
 '사실상 아바타 2.5가 아니었나, 영화 <아바타: 불과 재> 후기',
 '영화 아바타 불과 재 12세 관람가 러닝타임 등급 심사표 공개 수위',
 '아바타 3 불과 재 쿠키 영상 정보 영화 리뷰',
 '영화 아바타3: 불과 재(2025) 정보, 줄거리, 결말, 후기',
 '영화 아바타 3편 불과 재 줄거리 결말 쿠키없음',
 '아바타 불과재 출격! 12월 셋째주 개봉 예정 영화',
 '[영화 리뷰] 아바타: 불과 재 (2025년) 감상평과 후기, 영화 정보... ',
 "영화 '아바타: 불과 재' 시사회 초청 이벤트, KT 멤버십 X 롯데시네마"]

In [47]:
total=[blog_lst,blog_title]

[['https://blog.naver.com/syunni1225/224115606045',
  'https://blog.naver.com/kaminari2002/224114898643',
  'https://blog.naver.com/seok9c/224115025215',
  'https://blog.naver.com/duswns202/224106859517',
  'https://blog.naver.com/dudghkrkwhgdms/224115758710',
  'https://blog.naver.com/blac_beat/224128473123',
  'https://blog.naver.com/realnogun/224113577463',
  'https://blog.naver.com/c106507/224111497322',
  'https://blog.naver.com/sta-tio/224114494844',
  'https://blog.naver.com/stmisn/224091045516'],
 ['아바타: 불과 재(Avatar: Fire and Ash, 2025) 영화 정보... ',
  '<아바타3: 불과 재> 4Dx3D후기 :: 그래도 극장에서 봐야하는 영화.... ',
  '사실상 아바타 2.5가 아니었나, 영화 <아바타: 불과 재> 후기',
  '영화 아바타 불과 재 12세 관람가 러닝타임 등급 심사표 공개 수위',
  '아바타 3 불과 재 쿠키 영상 정보 영화 리뷰',
  '영화 아바타3: 불과 재(2025) 정보, 줄거리, 결말, 후기',
  '영화 아바타 3편 불과 재 줄거리 결말 쿠키없음',
  '아바타 불과재 출격! 12월 셋째주 개봉 예정 영화',
  '[영화 리뷰] 아바타: 불과 재 (2025년) 감상평과 후기, 영화 정보... ',
  "영화 '아바타: 불과 재' 시사회 초청 이벤트, KT 멤버십 X 롯데시네마"]]

In [ ]:
blog_content=[]
for i in blog_lst:
    url_tmp=i
    new=req.get(url_tmp)
    soup= bs(new.text,'lxml')
    
blog_lst   

### iframe
- 하나의 페이지에 많은 양의 데이터가 저장되어있으면 페이지가 무거워짐 -> 속도가 느려짐
- 데이터는 다른 저장사이트에 저장해두고 현재 페이지엣는 보여주만 하는형태
- 데이터 수집을 위해서 시렞 데이터가 저장된 공간으로 접근
- ex)블로그,카페 ,지식인,지도
- 

- 실제 데이터의 주소
  - 메인 주소값 + src -> 실제 블로그 주소 생성
  - https://blog.naver.com/ + src

In [95]:

new=req.get(blog_lst[0])
soup1= bs(new.text,'lxml')


In [119]:
a=soup1.select_one('#mainFrame').get('src')

In [122]:
blog_url='https://blog.naver.com/'+a
blog_url

'https://blog.naver.com//PostView.naver?blogId=syunni1225&logNo=224115606045&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false'

In [128]:
blog_url=[]
for i in blog_lst:
    new=req.get(i)
    soup1= bs(new.text,'lxml')
    a=soup1.select_one('#mainFrame').get('src')
    tmp='https://blog.naver.com/'+a
    blog_url.append(tmp)
blog_url    

['https://blog.naver.com//PostView.naver?blogId=syunni1225&logNo=224115606045&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false',
 'https://blog.naver.com//PostView.naver?blogId=kaminari2002&logNo=224114898643&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false',
 'https://blog.naver.com//PostView.naver?blogId=seok9c&logNo=224115025215&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false',
 'https://blog.naver.com//PostView.naver?blogId=duswns202&logNo=224106859517&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false',
 'https://blog.naver.com//PostView.naver?blogId=dudghkrkwhgdms&logNo=224115758710&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false',
 'https://blog.naver.com//PostView.naver?blogId=blac_beat&logNo=224128473123&redirect=Dlog&widgetTypeCall=true&noTrackingCode=true&directAccess=false',
 'https://blog.naver.com//PostView.naver?blogId=realnogun&logNo=224113577463&redir

In [132]:
new=req.get(blog_url[0])
soup2= bs(new.text,'lxml')

In [137]:
soup2.select_one('.se-module.se-module-text.se-title-text>p>span').text

'아바타: 불과 재(Avatar: Fire and Ash, 2025) 영화 정보, 리뷰ː데우스 엑스 마키나적 해결법'

In [172]:
a=soup2.select('.se-main-container')[0].text

In [173]:
a

"\n\n\n\n\n영화 <아바타: 불과 재> 정보, 줄거리, 리뷰\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\u200b\u200b\u200b\n\n\n\n\n \n\n\n\n영화 정보\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n기본 정보\n\n\n\n\n\n\n\n\n원제 : Avatar: Fire and Ash수입/배급 : 월트디즈니 컴퍼니 코리아감독 : 제임스 카메론출연 : 샘 워싱턴, 조이 살다나, 시고니 위버, 스티븐 랭, 우나 채플린, 케이트 윈슬렛 외제작 국가 : 미국장르 : 액션, 어드벤처(모험), 판타지, SF, 스릴러관람 등급 : 12세 이상 관람가상영 시간 : 195분극장 개봉 : 2025년 12월 17일\u200b\n\n\n\n\n \n\n\n\n줄거리\n\n\n\n\n\n\n\n\n큰아들을 잃은 제이크 설리와 네이티리 가족은 저마다 애도를 하고 있다. 그러던 중 그들 가족에게 다시 위기가 찾아오는데...\u200b\u200b\u200b\n\n\n\n\n \n\n\n\n리뷰로 보는 관람 포인트\n\n\n\n\n\n\n\n\n여전히 말 안 듣는 아이들\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n3편을 보기 위해 굳이 2편을 복습하지 않아도 될 것 같아 일단 보기 시작한 영화 오프닝에서는 전편에서 사망한 네테이얌이 등장해 로아크와 자유롭게 비행을 하는 모습을 보여줬다. 나비족만이 할 수 있는 의식 같은 걸로 로아크는 죽은 형과 대화하며 즐거운 시간을 보낼 수 있던 것이었다.로아크가 자신만의 방식으로 형을 그리워하고 있을 때 엄마인 네이티리는 상실감에 아무것도 하지 못하고 슬퍼하기만 했다. 그리고 설리는 인간이 다시금 공격을 해올 걸 예상해 바다에 가라앉은 인간들의 전함에서 총을 비롯한 각종 무기를 모으기 시작했다. 설리네 가족이 의지하고 있는 멧카이나의 부족장은 강하게 반발했지만 설리는 뜻을 굽히지 않았다.\u200b\n\n\n\n\n \n\

- 정규표현식을 활용한 데이터 전처리 ~
    - re
    - 정규표현식: 문자열내에서 특정 규칙을 찾아 변하는 도구

| 구성요소                  | 의미                       |
| --------------------- | ------------------------ |
| `r"..."`              | Raw string — 이스케이프 문자 무시 |
| `[^ ... ]`            | 괄호 안 문자 **제외한** 문자들      |
| `0-9`                 | 숫자                       |
| `가-힣`                 | 모든 한글                    |
| `?.!,¿`               | 허용할 문장 부호                |
| `+`                   | 하나 이상 연속된 구간             |
| `re.sub(..., " ", w)` | 매칭된 부분을 공백으로 치환          |

In [ ]:
blog_url=[]
for i in blog_lst:
    new=req.get(i)
    soup1= bs(new.text,'lxml')
    a=soup1.select_one('#mainFrame').get('src')
    tmp='https://blog.naver.com/'+a
    blog_url.append(tmp)
blog_url    

In [184]:
# 문자열 정처리 함수 -> 순사,문자 ()
def preprocess_sentence_kr(w):
  w = w.strip()
  w = re.sub(r"[^0-9가-힣?.!,¿a-zA-Z]+", " ", w) 
  w = w.strip() 
  return w
 